## installing libraries

In [1]:
!pip install mediapipe -q

In [2]:
!pip install opencv-python -q

## importing libraries

In [3]:
import numpy as np
import os
import mediapipe as mp
import cv2

## auxiliary functions

## main script

In [4]:
# MediaPipe solution for respectively drawing to the screen and hand landmark detection
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [5]:
# Object to interact with the webcam of your computer
webcam = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    
    while webcam.isOpened():

        # Read in frames while the webcam is open
        ret, frame = webcam.read()

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)
        
        # MediaPipe operates on the RGB color-scheme
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Apply the MediaPipe hand detection model on the frame
        results = hands.process(frame)
        
        # Convert back to the original BGR color-scheme
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
               
        # Drawing connected landmarks (joints)
        if results.multi_hand_landmarks:
            # Iterate over hands (only 1 in this case)
            for num, hand in enumerate(results.multi_hand_landmarks):
                # Draw the landmarks
                mp_drawing.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)
                # Print the position of landmark 8: the tip of the index finger
                print(results.multi_hand_landmarks[0].landmark[8])
        

        # Display the frame on the screen
        cv2.imshow('Catch The Dot!', frame)

        # Interrupt the webcam stream when users press the 'q' key (quit)          
        key = cv2.waitKey(100)
        if key == ord('q'):
            break

webcam.release()
cv2.destroyAllWindows()

I0000 00:00:1718272609.111601 1044032 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1718272609.120634 1044161 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718272609.129415 1044161 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/damon/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


x: 0.632347703
y: 0.337731063
z: -0.171990022

x: 0.588398576
y: 0.222423017
z: -0.097124286

x: 0.549754322
y: 0.161358804
z: -0.0832133144

x: 0.546002805
y: 0.188558251
z: -0.0682026744

x: 0.55132
y: 0.207245022
z: -0.0721532628

x: 0.553444
y: 0.210573316
z: -0.0679979697

x: 0.558460474
y: 0.208930641
z: -0.06598261

x: 0.55926168
y: 0.212136954
z: -0.0695501193

x: 0.498319805
y: 0.283514887
z: -0.0762407407

x: 0.422379017
y: 0.40437597
z: -0.0831446797

x: 0.409653842
y: 0.43929714
z: -0.0949419

x: 0.414114416
y: 0.440409601
z: -0.0984264165

x: 0.411495537
y: 0.43506518
z: -0.106033854

x: 0.41866523
y: 0.413403332
z: -0.107503489

x: 0.451158464
y: 0.348077029
z: -0.0986156

x: 0.503841698
y: 0.28268522
z: -0.10924416

x: 0.567625284
y: 0.222938
z: -0.124534093

x: 0.659064472
y: 0.18077001
z: -0.138010219

x: 0.718805969
y: 0.169168115
z: -0.161069453

x: 0.73012954
y: 0.167438865
z: -0.164975911

x: 0.726593256
y: 0.15465337
z: -0.155003518

x: 0.723278165
y: 0.15637666
z